In [1]:
import numpy as np
import torch
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
from skimage import color, exposure
from sklearn.pipeline import Pipeline
import torch
import torchvision.datasets as datasets
from torch.utils.data import Subset, DataLoader
import torchvision.transforms as transforms
from numpy.random import RandomState
import torchvision
import time
import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:

def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import random
import torchvision
from torchvision.models import resnet18
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
start_time = time.time()
# Set random seed for reproducibility
torch.manual_seed(0)
random.seed(0)

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
# Data augmentation during training:
transform_train = transforms.Compose([
                                    transforms.RandomCrop(32, padding=2, padding_mode='reflect'),
                                    transforms.RandomGrayscale(),
                                    # transforms.Resize((224, 224)),
                                    transforms.RandomHorizontalFlip(),
                                    torchvision.transforms.RandomAffine(degrees=30),
                                    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
                                    transforms.ToTensor(),
                                    normalize]) #careful to keep this one same
transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='./data',train=True, transform=transform_train, download=True)

#We need two copies of this due to weird dataset api
cifar_data_val = datasets.CIFAR10(root='./data',train=True, transform=transform_val, download=True)


resnet_architecture_accs = []

# Select random classes
classes = random.sample(range(10), 2)

for seed in range(1, 5):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))

  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128,
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128,
                                           shuffle=False)


  # Define ResNet model
  def create_resnet_model(num_classes):
    model = resnet18(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

  # Define hyperparameters for grid search
  params = {
    'lr': [0.001, 0.0001],
    'batch_size': [4, 8],
    'num_epochs': [5, 10]
  }

  # Initialize ResNet model
  resnet_architecture_model = create_resnet_model(num_classes=10)
  resnet_architecture_model = resnet_architecture_model.to(device)
  # model.to(device)
  optimizer = torch.optim.SGD(resnet_architecture_model.parameters(),lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
  for epoch in range(100):
    train(resnet_architecture_model, device, train_loader, optimizer, epoch, display=epoch%5==0)

  resnet_architecture_accs.append(test(resnet_architecture_model, device, val_loader))

resnet_architecture_accs = np.array(resnet_architecture_accs)
print('Acc over 5 instances: %.2f +- %.2f'%(resnet_architecture_accs.mean(),resnet_architecture_accs.std()))
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train Epoch: 0 [0/50 (0%)]	Loss: 2.429064
Train Epoch: 5 [0/50 (0%)]	Loss: 0.717253
Train Epoch: 10 [0/50 (0%)]	Loss: 0.584743
Train Epoch: 15 [0/50 (0%)]	Loss: 0.495980
Train Epoch: 20 [0/50 (0%)]	Loss: 0.804601
Train Epoch: 25 [0/50 (0%)]	Loss: 0.481059
Train Epoch: 30 [0/50 (0%)]	Loss: 0.485072
Train Epoch: 35 [0/50 (0%)]	Loss: 0.553778
Train Epoch: 40 [0/50 (0%)]	Loss: 0.366611
Train Epoch: 45 [0/50 (0%)]	Loss: 0.429497
Train Epoch: 50 [0/50 (0%)]	Loss: 0.423182
Train Epoch: 55 [0/50 (0%)]	Loss: 0.419541
Train Epoch: 60 [0/50 (0%)]	Loss: 0.202320
Train Epoch: 65 [0/50 (0%)]	Loss: 0.522186
Train Epoch: 70 [0/50 (0%)]	Loss: 0.256521
Train Epoch: 75 [0/50 (0%)]	Loss: 0.158491
Train Epoch: 80 [0/50 (0%)]	Loss: 0.183203
Train Epoch: 85 [0/50 (0%)]	Loss: 0.199460
Train Epoch: 90 [0/50 (0%)]	Loss: 0.274679
Train Epoch: 95 [0/50 (0%)]	Loss: 0.256618


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 1.4114, Accuracy: 275/400 (68.75%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.141577
Train Epoch: 5 [0/50 (0%)]	Loss: 0.641993
Train Epoch: 10 [0/50 (0%)]	Loss: 0.698635
Train Epoch: 15 [0/50 (0%)]	Loss: 0.537210
Train Epoch: 20 [0/50 (0%)]	Loss: 0.591389
Train Epoch: 25 [0/50 (0%)]	Loss: 0.603619
Train Epoch: 30 [0/50 (0%)]	Loss: 0.356911
Train Epoch: 35 [0/50 (0%)]	Loss: 0.368988
Train Epoch: 40 [0/50 (0%)]	Loss: 0.304884
Train Epoch: 45 [0/50 (0%)]	Loss: 0.303637
Train Epoch: 50 [0/50 (0%)]	Loss: 0.382197
Train Epoch: 55 [0/50 (0%)]	Loss: 0.302598
Train Epoch: 60 [0/50 (0%)]	Loss: 0.469878
Train Epoch: 65 [0/50 (0%)]	Loss: 0.280594
Train Epoch: 70 [0/50 (0%)]	Loss: 0.156310
Train Epoch: 75 [0/50 (0%)]	Loss: 0.245715
Train Epoch: 80 [0/50 (0%)]	Loss: 0.227400
Train Epoch: 85 [0/50 (0%)]	Loss: 0.084488
Train Epoch: 90 [0/50 (0%)]	Loss: 0.219385
Train Epoch: 95 [0/50 (0%)]	Loss: 0.186873

Test set: Average loss: 1.47